# Deeplabv3 Segmentaion

## Dataset
gdown ID 
- Binary 
  - Resize 512x512, padding Original : 1EcAeCLDTlHXf9jzBiSTbqIHMSF6WZC03
  - Augmentation Train : 1weHJqe9aGPH1I6bP1K7FzRglIcFfW1fH
  - Augmentation Valid : 17xTOcDDZPlmZTz4kn1oTozjt_mdulrNz



## Original dataset download and train

### Git clone

In [ ]:
cd /content

In [ ]:
!git clone https://github.com/jcjang1/CodeEnter_DeepLabv3FineTuning.git

### Dataset download

In [ ]:
!gdown 1EcAeCLDTlHXf9jzBiSTbqIHMSF6WZC03

In [ ]:
!unzip /content/512_padding_data_220722.zip

### Make folder
- 원하는 데이터 class 번호만 바꿔주면서 작업
- ex)

```
!mv {class 번호} data_set
```



In [ ]:
cd /content/make_data

In [ ]:
!mv 4 data_set
!mv ./data_set/images ./data_set/Images
!mv ./data_set/masks ./data_set/Masks

### Trainning

In [ ]:
cd /content/CodeEnter_DeepLabv3FineTuning

In [ ]:
!python main.py --data-directory /content/make_data/data_set --exp_directory CFExp

## Augmentation dataset download and train

### Git clone

In [1]:
cd /content

/content


In [2]:
!git clone https://github.com/jcjang1/CodeEnter_DeepLabv3FineTuning.git

Cloning into 'CodeEnter_DeepLabv3FineTuning'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 384 (delta 20), reused 26 (delta 10), pack-reused 340
Receiving objects: 100% (384/384), 4.22 MiB | 15.28 MiB/s, done.
Resolving deltas: 100% (41/41), done.


### Aug dataset download

In [3]:
!gdown 1weHJqe9aGPH1I6bP1K7FzRglIcFfW1fH
!gdown 17xTOcDDZPlmZTz4kn1oTozjt_mdulrNz

Downloading...
From: https://drive.google.com/uc?id=1weHJqe9aGPH1I6bP1K7FzRglIcFfW1fH
To: /content/DeepLabv3_Binary_AUGtrain_data.zip
100% 107M/107M [00:02<00:00, 38.6MB/s] 
Downloading...
From: https://drive.google.com/uc?id=17xTOcDDZPlmZTz4kn1oTozjt_mdulrNz
To: /content/DeepLabv3_Binary_AUGvalid_data.zip
100% 31.7M/31.7M [00:00<00:00, 159MB/s] 


### Make folder
- 원하는 데이터 class 번호만 바꿔주면서 작업
- ex)

```
!mv /content/aug_data/{class번호} /content/aug_data/Train
!mv /content/test_data/{class번호} /content/test_data/Test
```



In [4]:
!mkdir aug_data_set

In [ ]:
!unzip /content/DeepLabv3_Binary_AUGtrain_data.zip
!unzip /content/DeepLabv3_Binary_AUGvalid_data.zip

In [9]:
!mv /content/aug_data/4 /content/aug_data/Train
!mv /content/test_data/4 /content/test_data/Test

!mv /content/aug_data/Train/images /content/aug_data/Train/Images
!mv /content/aug_data/Train/masks /content/aug_data/Train/Masks

!mv /content/test_data/Test/images /content/test_data/Test/Images
!mv /content/test_data/Test/masks /content/test_data/Test/Masks

In [10]:
!cp -r /content/aug_data/Train ./aug_data_set
!cp -r /content/test_data/Test ./aug_data_set

### Trainning

In [11]:
cd /content/CodeEnter_DeepLabv3FineTuning

/content/CodeEnter_DeepLabv3FineTuning


In [ ]:
!python sep_main.py --data-directory /content/aug_data_set --exp_directory CFExp

## Inference

In [ ]:
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [ ]:
# Load the trained model 
model = torch.load('./CFExp/weights.pt')
# Set the model to evaluate mode
model.eval()

### Visualizing

In [ ]:
df = pd.read_csv('./CFExp/log.csv')
df[df['Test_loss'] == df['Test_loss'].min()]

In [ ]:
drop_loss = df.drop(columns=['Train_loss', 'Test_loss'], axis=1)
loss_df = df[df.columns[:3]]

In [ ]:
column_list = list(drop_loss.columns[1:])
color_list = ['red', 'orange', 'black', 'green', 'blue', 'purple']

#### Show log graph

In [ ]:
fig = plt.figure(figsize=(16, 8))


ax = fig.add_subplot(1, 2, 1)
for color, column in zip(color_list, column_list):
    ax.plot(drop_loss[column], color=color, label=column)
ax.title.set_text("Score")
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.plot(loss_df["Train_loss"], color="red", label="Train_loss")
ax.plot(loss_df["Test_loss"], color="blue", label="Test_loss")
ax.title.set_text("Loss")
ax.legend()

plt.show()

#### Show bar plot

In [ ]:
# Read  a sample image and mask from the data-set
img = cv2.imread('/content/test_data/Test/Images/b_0171.jpg')
img_plot = img.transpose(2,0,1).reshape(1,3,img.shape[0], img.shape[1])
mask = cv2.imread('/content/test_data/Test/Masks/b_0171.jpg')
with torch.no_grad():
    a = model(torch.from_numpy(img_plot).type(torch.cuda.FloatTensor)/255)

In [ ]:
plt.hist(a['out'].data.cpu().numpy().flatten())

#### Show image

In [ ]:
# Plot the input image, ground truth and the predicted output
plt.figure(figsize=(10,10));
plt.subplot(131);
plt.imshow(img_plot[0,...].transpose(1,2,0));
plt.title('Image')
plt.axis('off');
plt.subplot(132);
plt.imshow(mask);
plt.title('Ground Truth')
plt.axis('off');
plt.subplot(133);
plt.imshow(a['out'].cpu().detach().numpy()[0][0]>0.2);
plt.title('Segmentation Output')
plt.axis('off');
# plt.savefig('./CFExp/SegmentationOutput.png',bbox_inches='tight')

In [ ]:
plt.imshow(img)
plt.imshow(a['out'].cpu().detach().numpy()[0][0]>0.2, alpha=0.4);
plt.title('Prediction')
plt.show()